<a href="https://colab.research.google.com/github/pirroux/Tennis_Shot_Identification/blob/master/shot_identification_works.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/pirroux/Tennis_Shot_Identification.git

Cloning into 'Tennis_Shot_Identification'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 0), reused 24 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 90.67 KiB | 1.74 MiB/s, done.


In [1]:
%cd Tennis_Shot_Identification/

/content/Tennis_Shot_Identification


In [5]:
!cp /content/drive/MyDrive/uploads_for_tennis/mmodelfinal.h5 /content/drive/MyDrive/uploads_for_tennis/yolov7-w6-pose.pt /content/drive/MyDrive/uploads_for_tennis/RG_crop.mp4 .

In [4]:
!python -m pip install pillow --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 18.3 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: Pillow 9.4.0
    Uninstalling Pillow-9.4.0:
      Successfully uninstalled Pillow-9.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.31.6 requires pillow<10.1.0,>=8.3.2, but you have pillow 10.2.0 which is incompatible.


In [4]:
#**Importing All the Required Libraries***
import cv2
from google.colab.patches import cv2_imshow
import time
import torch
import argparse
import numpy as np
from utils.datasets import letterbox
from utils.torch_utils import select_device
from models.experimental import attempt_load
from utils.plots import output_to_keypoint, plot_skeleton_kpts, plot_one_box_kpt, colors
from utils.general import non_max_suppression_kpt, strip_optimizer
from torchvision import transforms
import tensorflow
from PIL import ImageFont, ImageDraw, Image

#Creating an Empty Dictionary, to save the to shot count of each type of stroke
# The first element in the dictionary is the  key, which contains the shot type
# The second element in the dictionary is the value, which contains the shot count of each of the stroke
# for example ForeHandGroundStroke is played by the player 10 times
# while the Backhand Ground Stroke is played by the player 4 times.


object_counter = {}

#Creating a Function  by the name load_classes,using this function we will load the coco.names file and read each
#of the object name, in the coco.names file and this function will return all the object names in the coco.names
#file in the form of a list
def load_classes(path):
    # Loads *.names file at 'path'
    with open(path, 'r') as f:
        names = f.read().split('\n')
    return list(filter(None, names))  # filter removes empty strings (such as last line)

#This is the main function, here we are passing the yolov7 pose weights, by default we are setting the deivce as CPU
#setting the line thick ness of the skeleton lines as well

@torch.no_grad()
def run(poseweights='yolov7-w6-pose.pt', source='pose.mp4', device='cpu', names = 'utils/coco.names', line_thickness = 2):

    path = source
    ext = path.split('/')[-1].split('.')[-1].strip().lower()
    if ext in ["mp4", "webm", "avi"] or ext not in ["mp4", "webm", "avi"] and ext.isnumeric():
        input_path = int(path) if path.isnumeric() else path
        device = select_device(device)
        names = load_classes(names)
        half = device.type != 'cpu'
        model = attempt_load(poseweights, map_location=device)
        _ = model.eval()

        cap = cv2.VideoCapture(input_path)
        webcam = False

        if (cap.isOpened() == False):
            print('Error while trying to read video. Please check path again')

        fw, fh = int(cap.get(3)), int(cap.get(4))
        if ext.isnumeric():
            webcam = True
            fw, fh = 1280, 768
        vid_write_image = letterbox(
            cap.read()[1], (fw), stride=64, auto=True)[0]

        resize_height, resize_width = vid_write_image.shape[:2]
        out_video_name = "output" if path.isnumeric(
        ) else f"{input_path.split('/')[-1].split('.')[0]}"
        out = cv2.VideoWriter(f"{out_video_name}_wed.mp4", cv2.VideoWriter_fourcc(
            *'mp4v'), 30, (resize_width, resize_height))
        if webcam:
            out = cv2.VideoWriter(f"{out_video_name}_mon.mp4", cv2.VideoWriter_fourcc(
                *'mp4v'), 30, (fw, fh))

        frame_count, total_fps = 0, 0

        # =3.0===Load custom font ===========
        fontpath = "sfpro.ttf"
        font = ImageFont.truetype(fontpath, 32)
        # ===================================

        # ==4.0=== Load trained pose-indentification model======
        tf_model = tensorflow.keras.models.load_model('mmodelfinal.h5')
        # ==================================================

        # == 5.0 == variable declaration===========
        sequence = []
        keypoints = []
        pose_name = ''
        posename_list = []
        actions = np.array(['BackHand-GroundStroke', 'Forehand-GroundStroke'])
        label_map = {label: num for num, label in enumerate(actions)}
        j = 1
        seq = 30
        # =============================================
        while cap.isOpened:

            print(f"Frame {frame_count} Processing")
            ret, frame = cap.read()
            if ret:
                #First, we will do for the Player 2
                origimage = frame
                #Creating a Black Mask
                mask=np.zeros(frame.shape[:2] , dtype="uint8")
                #Setting the ROI for the Player 2
                roi = cv2.rectangle(mask, (16, 415), (1274, 714),(255,255,255), -1)
                #Overlapping the Mask on the Original Image
                masked=cv2.bitwise_and(frame,frame,mask=mask)
                masked[np.where((masked==[0,0,0]).all(axis=2))]=[255,0,0]
                orig_image = masked

                # preprocess image
                image_ = cv2.cvtColor(origimage, cv2.COLOR_BGR2RGB)
                image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2RGB)
                image_ = letterbox(image_, (resize_width), stride=64, auto=True)[0]

                image = letterbox(image, (resize_width), stride=64, auto=True)[0]
                image__ = image_.copy()
                image___ = image.copy()
                image_ = transforms.ToTensor()(image_)
                image = transforms.ToTensor()(image)
                image_ = torch.tensor(np.array([image_.numpy()]))
                image = torch.tensor(np.array([image.numpy()]))
                image_ = image_.to(device)
                image_ = image_.float()
                image = image.to(device)
                image = image.float()
                start_time = time.time()


                ###Player1 Starts From Here
                origimage2 = frame
                #Creating a Black Mask
                mask2=np.zeros(frame.shape[:2] , dtype="uint8")
                #Setting the Region of Interest for the Player 1
                roi = cv2.rectangle(mask2, (348, 133), (1033, 356),(255,255,255), -1)
                #Overlapping the Mask on the Original Image
                masked2=cv2.bitwise_and(frame,frame,mask=mask2)
                masked2[np.where((masked2==[0,0,0]).all(axis=2))]=[255,0,0]
                orig_image2 = masked2

                # preprocess image
                image_2 = cv2.cvtColor(origimage2, cv2.COLOR_BGR2RGB)
                image2 = cv2.cvtColor(orig_image2, cv2.COLOR_BGR2RGB)

                image_2 = letterbox(image_2, (resize_width), stride=64, auto=True)[0]

                image2 = letterbox(image2, (resize_width), stride=64, auto=True)[0]
                image__2 = image_2.copy()
                image___2 = image2.copy()
                image_2 = transforms.ToTensor()(image_2)
                image2 = transforms.ToTensor()(image2)
                image_2 = torch.tensor(np.array([image_2.numpy()]))
                image2 = torch.tensor(np.array([image2.numpy()]))
                image_2 = image_2.to(device)
                image_2 = image_2.float()
                image2 = image2.to(device)
                image2 = image2.float()
                start_time = time.time()

                with torch.no_grad():
                    output, _ = model(image)
                    output2, _ = model(image2)
                output_data = non_max_suppression_kpt(output, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
                output = output_to_keypoint(output_data)
                output_data2 = non_max_suppression_kpt(output2, 0.25, 0.65, nc=model.yaml['nc'], nkpt=model.yaml['nkpt'], kpt_label=True)
                output2 = output_to_keypoint(output_data2)

                img_ = image_[0].permute(1, 2, 0) * 255
                img_ = img_.cpu().numpy().astype(np.uint8)

                img_img = cv2.cvtColor(img_, cv2.COLOR_RGB2BGR)

                img = image[0].permute(1, 2, 0) * 255
                img = img.cpu().numpy().astype(np.uint8)

                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
                img_2 = image_2[0].permute(1, 2, 0) * 255
                img_2 = img_2.cpu().numpy().astype(np.uint8)

                img_img2 = cv2.cvtColor(img_2, cv2.COLOR_RGB2BGR)
                height, width,_ = img_img2.shape
                img2 = image2[0].permute(1, 2, 0) * 255
                img2 = img2.cpu().numpy().astype(np.uint8)

                img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
                #Plotting the Skeleton Keypoints on the Player 1
                for idx in range(output2.shape[0]):
                    plot_skeleton_kpts(img_img2, output2[idx, 7:].T, 3)

                gn = torch.tensor(img.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                for i, pose in enumerate(output_data):  # detections per image

                    if len(output_data):  #check if no pose
                        for c in pose[:, 5].unique(): # Print results
                            n = (pose[:, 5] == c).sum()  # detections per class
                            print("No of Objects in Current Frame : {}".format(n))

                        for det_index, (*xyxy, conf, cls) in enumerate(reversed(pose[:,:6])): #loop over poses for drawing on frame
                            c = int(cls)  # integer class
                            kpts = pose[det_index, 6:]
                            label = names[c]
                            #This function will create a bounding box and assign a label to the player 2
                            plot_one_box_kpt(xyxy, img_img2, label=label, color=colors(c, True),
                                        line_thickness=line_thickness, kpts=kpts, steps=3,
                                        orig_shape=img.shape[:2])
                            # == 6.0 === preprocess model input data and pose prediction =======
                            #1.So now, each video frame was fed into the YOLOv7 Pose Estimation Model
                            #and predicted Key points Landmarks (X-coordinate, Y-coordinate and confidence) were extracted
                            # 2. and stacked together as a sequence of 30 frames.
                            #Here we will plot the skeleton keypoints on the Player 2

                            if j <= seq:
                                for idx in range(output.shape[0]):
                                    kpts = output[idx, 7:].T
                                    plot_skeleton_kpts(img_img2, kpts, 3)
                                    sequence.append(kpts.tolist())

                            # So if the length of the sequence is 30, then we will do the shot prediction,
                            # that whether it is a forehand ground stroke or the backhand ground stroke
                            if len(sequence) == 30:
                                # Doing the Shot Prediction over here
                                result = tf_model.predict(np.expand_dims(sequence, axis=0))
                                pose_name = actions[np.argmax(result)]
                                #So we will save all the sequence values which we have got
                                keypoints.append(sequence)
                                #We are also saving all the Shot Names in a list as well
                                posename_list.append(pose_name)
                                print(sequence)
                                print(keypoints)
                                print(pose_name)
                                print(posename_list)
                                # So here we are saying if the shot name is not in the object counter dictionary
                                #then add the name
                                #If the shot name is already there in the object counter dictionary then just
                                #increment the counter
                                if pose_name not in object_counter:
                                  object_counter[pose_name] = 1
                                else:
                                  object_counter[pose_name] += 1
                            #And when the value of j becomes equal to the value of the sequence
                            #which we have the defined as 30, then remove all the previous values of sequence
                            #and set it as an empty list and start processing on the next frames

                            if j == seq:
                                sequence = []
                                j = 0
                            j += 1
                # =============================================================
                xstart = (fw//2)
                ystart = (fh-100)
                yend = (fh-50)
                # So After we have the shot name and the total shot count of each shot, we want to diplay them
                # in the output video so here we are just setting the UI where we want to display the Shot names and
                #the total shot count in the Output Video
                # = 7.0 == Draw prediction ==================================
                if pose_name == "BackHand-GroundStroke":
                    cv2.line(img_img2, ((width - (width-50)),25), ((width - (width-200)),25), [85,45,255], 40)
                    cv2.putText(img_img2, "Shot Type", ((width - (width-50)),35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
                    cv2.line(img_img2, ((width - (width-50)),75), ((width - (width-500)),75), [85,45,255], 40)
                    cv2.putText(img_img2, pose_name, ((width - (width-50)),85), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)

                elif pose_name == "Forehand-GroundStroke":
                    cv2.line(img_img2, ((width - (width-50)),25), ((width - (width-200)),25), [85,45,255], 40)
                    cv2.putText(img_img2, "Shot Type", ((width - (width-50)),35), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
                    cv2.line(img_img2, ((width - (width-50)),75), ((width - (width-500)),75), [85,45,255], 40)
                    cv2.putText(img_img2, pose_name, ((width - (width-50)),85), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)

                # ====================================================================
                for idx, (key, value) in enumerate(object_counter.items()):
                    cnt_str = str(key) + ":" +str(value)
                    cv2.line(img_img2, ((width-500),45), (width,45), [85,45,255], 40)

                    cv2.putText(img_img2, f'Total Shot Count', ((width-500),55), 0, 1, [225, 255, 255], thickness=2, lineType=cv2.LINE_AA)
                    cv2.line(img_img2, ((width-500), 85 + (idx*40)), (width, 85 + (idx*40)), [85, 45, 255], 30)
                    cv2.putText(img_img2, cnt_str, ((width-500), 95 + (idx*40)), 0, 1, [255, 255, 255], thickness = 2, lineType = cv2.LINE_AA)
                # display image
                # if webcam:
                #     cv2.imshow("Detection", img)
                #     key = cv2.waitKey(1)
                #     if key == ord('c'):
                #         break
                # else:
                #     img_ = img.copy()
                #     img_ = cv2.resize(
                #         img_, (960, 540), interpolation=cv2.INTER_LINEAR)
                #     cv2.imshow("Detection", img_img2)
                #     cv2.waitKey(1)

                end_time = time.time()
                fps = 1 / (end_time - start_time)
                total_fps += fps
                frame_count += 1
                out.write(img_img2)
            else:
                break

        cap.release()
        avg_fps = total_fps / frame_count
        print(f"Average FPS: {avg_fps:.3f}")


# def parse_opt():
#     parser = argparse.ArgumentParser()
#     parser.add_argument('--poseweights', nargs='+', type=str,
#                         default='yolov7-w6-pose.pt', help='model path(s)')
#     parser.add_argument('--source', type=str,
#                         help='path to video or 0 for webcam')
#     parser.add_argument('--device', type=str, default='cpu',
#                         help='cpu/0,1,2,3(gpu)')
#     parser.add_argument('--line_thickness', default = 3, help = 'Please Input the Value of Line Thickness')

#     opt = parser.parse_args()
#     return opt


# def main(opt):
#     run(**vars(opt))


# if __name__ == "__main__":
#     opt = parse_opt()
#     strip_optimizer(opt.device, opt.poseweights)
#     main(opt)


In [5]:
run(source="/content/Tennis_Shot_Identification/RG_crop.mp4", device="0")

Fusing layers... 


Frame 0 Processing
No of Objects in Current Frame : 1
Frame 1 Processing
No of Objects in Current Frame : 1
Frame 2 Processing
No of Objects in Current Frame : 1
Frame 3 Processing
No of Objects in Current Frame : 1
Frame 4 Processing
No of Objects in Current Frame : 1
Frame 5 Processing
No of Objects in Current Frame : 1
Frame 6 Processing
No of Objects in Current Frame : 1
Frame 7 Processing
No of Objects in Current Frame : 1
Frame 8 Processing
No of Objects in Current Frame : 1
Frame 9 Processing
No of Objects in Current Frame : 1
Frame 10 Processing
No of Objects in Current Frame : 1
Frame 11 Processing
No of Objects in Current Frame : 1
Frame 12 Processing
No of Objects in Current Frame : 1
Frame 13 Processing
No of Objects in Current Frame : 1
Frame 14 Processing
No of Objects in Current Frame : 1
Frame 15 Processing
No of Objects in Current Frame : 1
Frame 16 Processing
No of Objects in Current Frame : 1
Frame 17 Processing
No of Objects in Current Frame : 1
Frame 18 Processing


On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   utils/plots.py

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	RG_crop.mp4
	RG_crop_wed.mp4
	mmodelfinal.h5

no changes added to commit (use "git add" and/or "git commit -a")


In [7]:
!git add utils/plots.py